In [356]:
en_path = '/Users/liuchu/linear_algebra_strang/translate/中英翻译数据集/train/news-commentary-v13.zh-en.en'
zh_path = '/Users/liuchu/linear_algebra_strang/translate/中英翻译数据集/train/news-commentary-v13.zh-en.zh'

In [357]:
def get_en_sentences():
    arr = []
    with open(en_path,'r') as f:
        count = 0
        for line in f.readlines():
            arr.append(line.strip())
    return arr

In [358]:
def get_zh_sentences():
    arr = []
    with open(zh_path,'r') as f:
        count = 0
        for line in f.readlines():
            arr.append(line.strip())
    return arr

In [359]:
en_sentences = get_en_sentences()
zh_sentences = get_zh_sentences()

# 制作批处理数据

In [360]:
en_sentences[:10]

['1929 or 1989?',
 'PARIS – As the economic crisis deepens and widens, the world has been searching for historical analogies to help us understand what has been happening.',
 'At the start of the crisis, many people likened it to 1982 or 1973, which was reassuring, because both dates refer to classical cyclical downturns.',
 'Today, the mood is much grimmer, with references to 1929 and 1931 beginning to abound, even if some governments continue to behave as if the crisis was more classical than exceptional.',
 'The tendency is either excessive restraint (Europe) or a diffusion of the effort (the United States).',
 'Europe is being cautious in the name of avoiding debt and defending the euro, whereas the US has moved on many fronts in order not to waste an ideal opportunity to implement badly needed structural reforms.',
 'For geo-strategists, however, the year that naturally comes to mind, in both politics and economics, is 1989.',
 'Of course, the fall of the house of Lehman Brothers 

In [361]:
zh_sentences[:10]

['1929 年 还是 1989 年 ?',
 '巴黎 - 随着 经济危机 不断 加深 和 蔓延 ， 整个 世界 一直 在 寻找 历史 上 的 类似 事件 希望 有助于 我们 了解 目前 正在 发生 的 情况 。',
 '一 开始 ， 很多 人 把 这次 危机 比作 1982 年 或 1973 年 所 发生 的 情况 ， 这样 得 类比 是 令人 宽心 的 ， 因为 这 两段 时期 意味着 典型 的 周期性 衰退 。',
 '如今 人们 的 心情 却是 沉重 多 了 ， 许多 人 开始 把 这次 危机 与 1929 年 和 1931 年 相比 ， 即使 一些 国家 政府 的 表现 仍然 似乎 把视 目前 的 情况 为 是 典型 的 而 看见 的 衰退 。',
 '目前 的 趋势 是 ， 要么 是 过度 的 克制 （ 欧洲 ）， 要么 是 努力 的 扩展 （ 美国 ）。',
 '欧洲 在 避免 债务 和 捍卫 欧元 的 名义 下正 变得 谨慎 ， 而 美国 已经 在 许多 方面 行动 起来 ， 以 利用 这一 理想 的 时机 来 实行 急需 的 结构性 改革 。',
 '然而 ， 作为 地域 战略 学家 ， 无论是 从 政治 意义 还是 从 经济 意义 上 ， 让 我 自然 想到 的 年份 是 1989 年 。',
 '当然 ， 雷曼 兄弟 公司 的 倒闭 和 柏林墙 的 倒塌 没有 任何 关系 。',
 '事实上 ， 从 表面 上 看 ， 两者 似乎 是 完全 是 相反 的 ： 一个 是 象征 着 压抑 和 人为 分裂 的 柏林墙 的 倒塌 ， 而 另 一个 是 看似 坚不可摧 的 并 令人 安心 的 金融 资本主义 机构 的 倒塌 。',
 '然而 ， 和 1989 年 一样 ， 2008 - 2009 年 很 可能 也 能 被 视为 一个 划时代 的 改变 ， 其 带来 的 发人深省 的 后果 将 在 几十年 后 仍 能 让 我们 感受 得到 。']

In [369]:
import re

def split_sentence_with_punctuation(sentence):
    # 使用正则表达式将句子分割成单词和标点符号
    words = re.findall(r'\w+|[^\w\s]', sentence)
    return words

# # 测试
# sentence = "PARIS – As the economic crisis deepens and widens, the world has been searching for historical analogies to help us understand what has been happening."
# word_list = split_sentence_with_punctuation(sentence)
# print(word_list)

In [372]:
zh_words_list = [split_sentence_with_punctuation(s) for s in zh_sentences]
en_words_list = [split_sentence_with_punctuation(s) for s in en_sentences]

In [376]:
zh_max_len = max(len(alist) for alist in zh_words_list)
en_mac_len = max(len(alist) for alist in en_words_list)



In [235]:
src_sentences = en_sentences
target_sentences = zh_sentences

In [391]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 编码器
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# 解码器
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = nn.functional.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [392]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

    decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS_token是起始符号的标记

    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:  # EOS_token是结束符号的标记
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [393]:
def tensorFromSentence(vocab, sentence):
    indices = [vocab[char] for char in sentence if char in vocab]  # Ensure char is in vocab
    indices.append(vocab['<eos>'])  # Append EOS token
    return torch.tensor(indices, dtype=torch.long, device=device).view(-1, 1)

def translate_sentence(sentence, encoder, decoder, input_vocab, output_vocab):
    input_tensor = tensorFromSentence(input_vocab, sentence)
    translated_sentence = translate(encoder, decoder, input_tensor, input_vocab, output_vocab)
    return translated_sentence

def translate(encoder, decoder, input_tensor, input_vocab, output_vocab, max_length=100):
    with torch.no_grad():
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()

        # Forward pass through encoder
        for ei in range(input_length):
            _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        # Decoder's first input is SOS token
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_word = output_vocab[topi.item()]
                decoded_words.append(decoded_word)

            decoder_input = topi.squeeze().detach()

        return ' '.join(decoded_words)

In [417]:
# 假设语料数据如下，这里我们使用了非常简单的例子
chinese_sentences = zh_words_list[:]
english_sentences = en_words_list[:]

# 为了简化问题，我们不使用词嵌入，而是直接根据字母或汉字进行索引
def create_dict(sentences):
    vocab = set(w for s in sentences for w in s)
    char_to_index = {char: i+1 for i, char in enumerate(vocab)}  # +1因为我们留出0作为padding
    char_to_index['<pad>'] = 0
    index_to_char = {i: char for char, i in char_to_index.items()}
    return char_to_index, index_to_char

chinese_vocab, chinese_index_vocab = create_dict(chinese_sentences)
chinese_vocab['<sos>'] = max(chinese_vocab.values()) + 1
chinese_index_vocab[chinese_vocab['<sos>']] = '<sos>'

english_vocab, english_index_vocab = create_dict(english_sentences)

# 将句子转换为索引
def sentence_to_indices(sentence, vocab):
    return [vocab[char] for char in sentence]

chinese_data = [sentence_to_indices(sentence, chinese_vocab) for sentence in chinese_sentences]
english_data = [sentence_to_indices(sentence, english_vocab) for sentence in english_sentences]

# 增加开始和结束标记

SOS_token = english_vocab['<sos>'] = max(english_vocab.values()) + 1
EOS_token = english_vocab['<eos>'] = max(english_vocab.values()) + 1
english_index_vocab[SOS_token] = '<sos>'
english_index_vocab[EOS_token] = '<eos>'
english_data = [[SOS_token] + sentence + [EOS_token] for sentence in english_data]

In [418]:
len(english_index_vocab),len(english_vocab)

(63875, 63875)

In [419]:
len(chinese_vocab)

92865

In [420]:
# 将数据转换为torch张量
def to_tensor(data):
    return torch.tensor(data, dtype=torch.long, device=device).view(-1, 1)

chinese_tensors = [to_tensor(data) for data in chinese_data]
english_tensors = [to_tensor(data) for data in english_data]

# 训练参数设置
num_epochs = 100

encoder = Encoder(len(chinese_vocab), 256).to(device)
decoder = Decoder(256, len(english_vocab)).to(device)



In [421]:

learning_rate = 0.001
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()


In [ ]:
# 训练循环
for epoch in range(num_epochs):
    total_loss = 0
    for ch_tensor, en_tensor in zip(chinese_tensors, english_tensors):
        loss = train(ch_tensor, en_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(chinese_sentences):.4f}')

In [402]:
zh_words_list[:10]

[['1929', '年', '还是', '1989', '年', '?'],
 ['巴黎',
  '-',
  '随着',
  '经济危机',
  '不断',
  '加深',
  '和',
  '蔓延',
  '，',
  '整个',
  '世界',
  '一直',
  '在',
  '寻找',
  '历史',
  '上',
  '的',
  '类似',
  '事件',
  '希望',
  '有助于',
  '我们',
  '了解',
  '目前',
  '正在',
  '发生',
  '的',
  '情况',
  '。'],
 ['一',
  '开始',
  '，',
  '很多',
  '人',
  '把',
  '这次',
  '危机',
  '比作',
  '1982',
  '年',
  '或',
  '1973',
  '年',
  '所',
  '发生',
  '的',
  '情况',
  '，',
  '这样',
  '得',
  '类比',
  '是',
  '令人',
  '宽心',
  '的',
  '，',
  '因为',
  '这',
  '两段',
  '时期',
  '意味着',
  '典型',
  '的',
  '周期性',
  '衰退',
  '。'],
 ['如今',
  '人们',
  '的',
  '心情',
  '却是',
  '沉重',
  '多',
  '了',
  '，',
  '许多',
  '人',
  '开始',
  '把',
  '这次',
  '危机',
  '与',
  '1929',
  '年',
  '和',
  '1931',
  '年',
  '相比',
  '，',
  '即使',
  '一些',
  '国家',
  '政府',
  '的',
  '表现',
  '仍然',
  '似乎',
  '把视',
  '目前',
  '的',
  '情况',
  '为',
  '是',
  '典型',
  '的',
  '而',
  '看见',
  '的',
  '衰退',
  '。'],
 ['目前',
  '的',
  '趋势',
  '是',
  '，',
  '要么',
  '是',
  '过度',
  '的',
  '克制',
  '（',
  '欧洲',
  '）',
  '，',
  '要

In [265]:
en_sentences[:10]

['1929 or 1989?',
 'PARIS – As the economic crisis deepens and widens, the world has been searching for historical analogies to help us understand what has been happening.',
 'At the start of the crisis, many people likened it to 1982 or 1973, which was reassuring, because both dates refer to classical cyclical downturns.',
 'Today, the mood is much grimmer, with references to 1929 and 1931 beginning to abound, even if some governments continue to behave as if the crisis was more classical than exceptional.',
 'The tendency is either excessive restraint (Europe) or a diffusion of the effort (the United States).',
 'Europe is being cautious in the name of avoiding debt and defending the euro, whereas the US has moved on many fronts in order not to waste an ideal opportunity to implement badly needed structural reforms.',
 'For geo-strategists, however, the year that naturally comes to mind, in both politics and economics, is 1989.',
 'Of course, the fall of the house of Lehman Brothers 

In [404]:
chinese_vocab

{'难': 1,
 '。': 2,
 '巴黎': 3,
 '看来': 4,
 '现在': 5,
 '脆弱': 6,
 '一个': 7,
 '压抑': 8,
 '一样': 9,
 '在': 10,
 '两段': 11,
 '整个': 12,
 '事件': 13,
 '正在': 14,
 '1973': 15,
 '类比': 16,
 '当然': 17,
 '事情': 18,
 '军备竞赛': 19,
 '政府': 20,
 '与': 21,
 '欧元': 22,
 '事实上': 23,
 '赢家': 24,
 '实行': 25,
 '一直': 26,
 '柏林墙': 27,
 '只是': 28,
 '类似': 29,
 '是': 30,
 '当今': 31,
 '世界': 32,
 '或许': 33,
 '每个': 34,
 '了': 35,
 '解体': 36,
 '积极': 37,
 '中国': 38,
 '他': 39,
 '意识形态': 40,
 '显示': 41,
 '对': 42,
 '债务': 43,
 '新': 44,
 '市场': 45,
 '战略': 46,
 '相反': 47,
 '把': 48,
 '然而': 49,
 '资本主义': 50,
 '其': 51,
 '明显': 52,
 '绝对': 53,
 '更': 54,
 '崩溃': 55,
 '改变': 56,
 '1931': 57,
 '集团': 58,
 '看': 59,
 '目前': 60,
 '了解': 61,
 '大多数': 62,
 '感受': 63,
 '似乎': 64,
 '民主': 65,
 '欧洲': 66,
 '一些': 67,
 '自由民主': 68,
 '后': 69,
 '信心': 70,
 '行动': 71,
 '周期性': 72,
 '寻找': 73,
 '沉重': 74,
 '如今': 75,
 '威胁': 76,
 '危机': 77,
 '或': 78,
 '想到': 79,
 '视为': 80,
 '来说': 81,
 '任何': 82,
 '宽心': 83,
 '完全': 84,
 '看似': 85,
 '经济危机': 86,
 '人们': 87,
 '让': 88,
 '重要': 89,
 '意义': 90,
 '1982': 91,
 '困境

In [409]:
zh_words_list[1][:5]

['巴黎', '-', '随着', '经济危机', '不断']

In [416]:
def translate_sentence(sentence):
    indices = sentence_to_indices(sentence, chinese_vocab)
    tensor = to_tensor(indices)
    translation = translate(encoder, decoder, tensor, chinese_index_vocab, english_index_vocab)
    return translation

# 翻译示例
print(translate_sentence(zh_words_list[2]))

<sos> At the start of the crisis , many people likened it to 1982 or 1973 , which was reassuring , because both dates refer to classical cyclical downturns . <EOS>


# batch size 训练

In [295]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 编码器
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)  # Set batch_first to True

    def forward(self, input, hidden):
        embedded = self.embedding(input)  # input: (batch_size, seq_len)
        output, hidden = self.gru(embedded, hidden)  # No need to permute
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

# 解码器
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)  # Set batch_first to True
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input)  # input: (batch_size,seq)
#         output = output.unsqueeze(1)  # Change to (batch_size, 1, hidden_size)
        output = nn.functional.relu(output)
        output, hidden = self.gru(output, hidden)  # No need to permute
        output = self.softmax(self.out(output.squeeze(1)))  # Squeeze to remove the sequence length dimension
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [311]:
import torch

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    batch_size = input_tensor.size(0)
    input_length = input_tensor.size(1)
    target_length = target_tensor.size(1)

    encoder_hidden = encoder.initHidden(batch_size)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    loss = 0

    # Encode the input sequence batch
    encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)

    # Prepare the initial decoder input (start with SOS tokens for each sequence in the batch)
    decoder_input = torch.tensor([[SOS_token] * batch_size], device=device).transpose(0, 1)  # Shape: (batch_size, 1)
    decoder_hidden = encoder_hidden

    # Assuming teacher forcing is not used here. If desired, it can be included with a certain probability.
    for di in range(target_length):
#         print(decoder_input.shape)
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.detach()  # Prepare next input
#         print(decoder_output.shape,target_tensor[:, di].shape,decoder_input.shape)

        loss += criterion(decoder_output, target_tensor[:, di])

        # Optionally, stop when all sequences in the batch reach EOS token
        if (decoder_input == EOS_token).all():
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / (target_length * batch_size)  # Normalize loss by total number of elements

In [312]:
import torch
import torch.nn as nn
import torch.optim as optim

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = 10  # Example vocabulary size for encoder
output_size = 10  # Example vocabulary size for decoder
hidden_size = 256
batch_size = 5
max_length = 7  # Maximum length of input and output sequences
epochs = 10

# Special tokens
SOS_token = 0  # Start-of-sequence token
EOS_token = 1  # End-of-sequence token

# Instantiate models
encoder = Encoder(input_size, hidden_size).to(device)
decoder = Decoder(hidden_size, output_size).to(device)

# Optimizers
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.01)

# Loss function
criterion = nn.NLLLoss()

In [313]:
def generate_fake_batch(batch_size, max_length, vocabulary_size):
    """ Generate a random batch of sequences for training. """
    # Random sequences of random lengths
    sequences = torch.randint(2, vocabulary_size, (batch_size, max_length)).to(device)
    return sequences

# Example of generating a batch
input_tensor = generate_fake_batch(batch_size, max_length, input_size)
target_tensor = generate_fake_batch(batch_size, max_length, output_size)

In [314]:
input_tensor

tensor([[2, 2, 4, 9, 8, 2, 7],
        [6, 6, 7, 6, 8, 2, 5],
        [9, 5, 6, 7, 3, 4, 4],
        [4, 5, 5, 5, 7, 7, 5],
        [3, 5, 6, 2, 5, 2, 2]])

In [315]:
def train_epoch(encoder, decoder, input_tensor, target_tensor):
    loss_total = 0  # Track loss

    for epoch in range(epochs):
        # Generate new data for each epoch
        input_tensor = generate_fake_batch(batch_size, max_length, input_size)
        target_tensor = generate_fake_batch(batch_size, max_length, output_size)

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length)
        loss_total += loss

        print(f'Epoch {epoch+1}, Loss: {loss:.4f}')

    print(f'Average Loss: {loss_total / epochs:.4f}')

# Call the training function
train_epoch(encoder, decoder, input_tensor, target_tensor)

Epoch 1, Loss: 0.4680
Epoch 2, Loss: 0.5614
Epoch 3, Loss: 0.4901
Epoch 4, Loss: 0.5791
Epoch 5, Loss: 0.5343
Epoch 6, Loss: 0.4939
Epoch 7, Loss: 0.4654
Epoch 8, Loss: 0.4840
Epoch 9, Loss: 0.4593
Epoch 10, Loss: 0.4287
Average Loss: 0.4964


# batch_len 版本

In [335]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

    def forward(self, input, hidden, input_lengths):
        embedded = self.embedding(input)  # input: (batch_size, seq_len)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, batch_first=True, enforce_sorted=False)
        packed_output, hidden = self.gru(packed, hidden)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)


class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input)  # input: (batch_size, 1)
        output = nn.functional.relu(output)
#         print('input shape, output shape 111',input.shape,output.shape)
        output, hidden = self.gru(output, hidden)
#         print('input shape, output shape  222',input.shape,output.shape)
        output = self.softmax(self.out(output.squeeze(1)))  # Make sure output is squeezed
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [336]:
def train(input_tensor, target_tensor, input_lengths, output_lengths, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    batch_size = input_tensor.size(0)
    encoder_hidden = encoder.initHidden(batch_size)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    loss = 0

    # 编码器处理输入
    encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden, input_lengths)

    # 解码器的初始输入
    decoder_input = torch.tensor([[SOS_token] * batch_size], device=device).transpose(0, 1)
    decoder_hidden = encoder_hidden

    # 解码器生成输出
    for di in range(max(output_lengths)):  # 使用最大输出长度
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        decoder_input = decoder_output.topk(1)[1].detach()

        for i in range(batch_size):
            if di < output_lengths[i]:  # 只计算有效长度内的损失
                loss += criterion(decoder_output[i], target_tensor[i, di])

        if (decoder_input == EOS_token).all():
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / sum(output_lengths)  # 按有效输出长度归一化损失

In [337]:
import torch
import torch.nn as nn
import torch.optim as optim

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 假设的词汇大小和隐藏层大小
input_size = 10
output_size = 10
hidden_size = 64

# 特殊符号定义
SOS_token = 0  # 句子开始符号
EOS_token = 1  # 句子结束符号

# 实例化模型
encoder = Encoder(input_size, hidden_size).to(device)
decoder = Decoder(hidden_size, output_size).to(device)

# 优化器
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.01)

# 损失函数
criterion = nn.NLLLoss()

# 创建一些虚拟数据
def generate_batch(batch_size):
    max_length = 10
    input_tensor = torch.LongTensor(batch_size, max_length).random_(2, input_size).to(device)
    target_tensor = torch.LongTensor(batch_size, max_length).random_(2, output_size).to(device)
    input_lengths = torch.LongTensor([random.randint(5, max_length) for _ in range(batch_size)]).to(device)
    target_lengths = torch.LongTensor([random.randint(5, max_length) for _ in range(batch_size)]).to(device)
    return input_tensor, target_tensor, input_lengths, target_lengths

batch_size = 5
input_tensor, target_tensor, input_lengths, target_lengths = generate_batch(batch_size)

In [338]:
# 训练模型
loss = train(input_tensor, target_tensor, input_lengths, target_lengths, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
print(f"Loss: {loss}")

Loss: 2.2624869346618652
